In [3]:
!pip install -qU langchain langchain-anthropic duckduckgo-search langchain-community

In [4]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("What is LangChain Tool Calling?")

'LangChain Tool Calling is a great way to connect your LLM to the information that LLM doesn\'t have, i.e to connect your LLM with the real world, the world that is still working and updating itself daily, making LangChain tool calling an essential tool for us to use with LLM. LangChain provides standard Tool Calling approach to many LLM providers like Anthropic, Cohere, Google, Mistral, and OpenAI support variants of this tool calling feature. bind_tools method What is a tool calling agent ? simply put a chain of langChain components(LLM, Tools, Propmt, Parsers) that utilize the LLM to repeatedly call itself in a loop. "Chains are great when we know the… Learn how to use tool calling functionality with various LLM providers that expose API\'s for reliable tool invocation. See the standard interface for attaching tool definitions, accessing tool calls, and creating tool calling agents with LangChain. Tool calling is a standout feature in agentic design, allowing the LLM to interact wit

In [ ]:
# Custom Tool for sending Slack messages using a webhook
import requests
import json
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def send_slack_message(message: str) -> str:
    """
    Sends a message to a Slack app using Slack webhook.

    Args:
        message (str): The message to send.

    Returns:
        str: A success or error message.
    """
    webhook_url = "https://YOUR_SLACK_WEBHOOK_URL"
    payload = {"text": message}
    headers = {"Content-type": "application/json"}

    try:
        response = requests.post(webhook_url, data=json.dumps(payload), headers=headers)
        response.raise_for_status()
        return "Message sent successfully!"
    except requests.exceptions.RequestException as e:
        return f"Error sending message: {e}"

message = "Hello from Claude!"
send_slack_message.invoke({"message": message})
# 'Message sent successfully!'

In [ ]:
from dotenv import load_dotenv
# take environment variables from .env
load_dotenv()

In [ ]:
from langchain_anthropic import ChatAnthropic

# ChatAnthropic - Claude 3 Haiku
llm = ChatAnthropic(model = "claude-3-haiku-20240307")

In [ ]:
# tools
tools = [search, send_slack_message]

# llm with tools
llm_with_tools = llm.bind_tools(tools)

In [ ]:
messages = [
    ("system", "You are a helpful assistant."),
    ("human", "What is LangChain Tool Calling?")
]

llm_with_tools.invoke(messages).tool_calls
#[{'name': 'duckduckgo_search',
#  'args': {'query': 'LangChain Tool Calling'},
#  'id': 'toolu_01Bfrz1Uhu84ggZd96Ae9De8'}]

messages = [
    ("system", "You are a helpful assistant."),
    ("human", "Send '👋 Happy Birthday from Claude! 😃' message to Slack app.")
]

llm_with_tools.invoke(messages).tool_calls
#[{'name': 'send_slack_message',
#  'args': {'message': '👋 Happy Birthday from Claude! 😃'},
#  'id': 'toolu_01LKxZvHpmDrdLooo1aJWqum'}]

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    ("system", "You are a helpful assistant."),
    ("human", "Send '👋 Happy Birthday from Claude! 😃' message to Slack app.")
]

ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"duckduckgo_search": search, "send_slack_message": send_slack_message}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages
#[('system', 'You are a helpful assistant.'),
#  ('human', "Send '👋 Happy Birthday from Claude! 😃' message to Slack app."),
#  AIMessage(content=[{'id': 'toolu_01HAV1YsmhUWK4ZRc8FJf7cL', 'input': {'message': '👋 Happy Birthday from Claude! 😃'}, 'name': 'send_slack_message', 'type': 'tool_use'}], response_metadata={'id': 'msg_01J59CnLZws1JngV9atfKwz9', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 499, 'output_tokens': 65}}, id='run-c9eb80b8-1b94-4e96-95a9-76f9f0c45c78-0', tool_calls=[{'name': 'send_slack_message', 'args': {'message': '👋 Happy Birthday from Claude! 😃'}, 'id': 'toolu_01HAV1YsmhUWK4ZRc8FJf7cL'}]),
#  ToolMessage(content='Message sent successfully!', tool_call_id='toolu_01HAV1YsmhUWK4ZRc8FJf7cL')]

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

# Prompt for creating Tool Calling Agent
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tool Calling Agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Run Agent
query = "What is LangChain Tool Calling Agent? Please send it to Slack app."
agent_executor.invoke({"input": query})